# Analyze results for 3D images
Jan 21, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_3d_image_analysis import *

# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
# import post_analysis_pandas as post

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [35]:
img_size=64
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/',
       '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/'}


### Read validation data and compute metrics

In [36]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'64':raw_data_dir+'3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy',
                 '128':raw_data_dir+'3d_data/dataset3_smoothing_const_params_128cube/val.npy'}

In [37]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
s_val=f_transform(s_val)
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

# del(s_val)

(1000, 64, 64, 64)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/modules_3d_image_analysis.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


### Read run data

In [38]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [39]:
u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)

In [40]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202105*')]
q=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(q)

interactive(children=(Dropdown(description='x', options=('20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_b…

In [58]:
result=q.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun


## Plot Losses

In [59]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [60]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
20583,20583.0,131.0,0.409867,0.409904,0.819771,3.029738,13.914418,10.884680,-0.958040,10.942328,-1.121989,NaN,NaN,2.097806,-1.244724,-2.977225,0.208711
20584,20584.0,131.0,0.344952,0.483194,0.828145,1.554166,12.188734,10.634567,-1.044578,10.881866,-0.929278,NaN,NaN,1.886886,-2.278840,-1.302487,0.206892
20585,20585.0,131.0,0.381237,0.444411,0.825648,3.338865,14.537067,11.198202,-1.288123,11.204821,-1.146666,NaN,NaN,2.084665,-1.481111,-3.300596,0.207391
20586,20586.0,131.0,0.393922,0.373581,0.767502,1.388705,12.096863,10.708158,-1.081605,11.073570,-0.979881,NaN,NaN,1.393416,-2.255419,-1.088401,0.228858
20587,20587.0,131.0,0.410266,0.374689,0.784954,3.073673,14.047178,10.973506,-0.785892,10.425389,-0.944247,NaN,NaN,1.879537,-1.157025,-3.020184,0.233304
20588,20588.0,131.0,0.352718,0.366556,0.719274,1.422298,12.151348,10.729051,-0.611679,10.715989,-0.480386,NaN,NaN,1.299463,-2.215254,-1.123391,0.206388
20589,20589.0,131.0,0.385430,0.408243,0.793672,3.214463,13.729593,10.515131,-0.573656,10.859622,-0.857559,NaN,NaN,2.029858,-1.292175,-3.172161,0.204575
20590,20590.0,131.0,0.391155,0.440124,0.831279,1.608834,12.551919,10.943086,-0.563219,11.099315,-0.336478,NaN,NaN,1.670003,-2.687477,-1.378783,0.207421
20591,20591.0,131.0,0.364267,0.402777,0.767043,3.173294,13.868661,10.695367,-0.829517,11.029449,-0.580830,NaN,NaN,2.222236,-1.255948,-3.126319,0.211713
20592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.959852,-0.782673,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [62]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.801232813000679


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
18459,18459.0,118.0,0.404792,0.335568,0.740360,3.424885,13.946660,10.521775,-1.833590,10.536683,-2.219496,NaN,NaN,1.930783,-1.522079,-3.384562,0.205610
18660,18660.0,119.0,0.363777,0.417354,0.781130,3.308839,13.238462,9.929623,-2.105474,10.590986,-2.197600,NaN,NaN,2.478998,-1.646247,-3.261307,0.206232
16814,16814.0,107.0,0.421975,0.443665,0.865639,0.868169,11.293832,10.425663,-2.051569,10.726904,-2.172974,NaN,NaN,1.419561,-2.305770,-0.265705,0.205558
18654,18654.0,119.0,0.342747,0.435356,0.778103,3.306589,14.107185,10.800596,-1.226143,10.618884,-2.166092,NaN,NaN,2.430929,-1.508596,-3.266417,0.209388
20239,20239.0,129.0,0.371107,0.588320,0.959427,2.893889,13.556862,10.662972,-2.170775,10.950659,-2.155584,NaN,NaN,2.042732,-0.534881,-2.830681,0.206788
20238,20238.0,129.0,0.402511,0.448501,0.851012,1.610276,12.418091,10.807816,-2.283474,10.823958,-2.151314,NaN,NaN,1.245454,-2.763181,-1.381527,0.212549
20165,20165.0,129.0,0.403449,0.411461,0.814910,1.949651,13.781869,11.832217,-1.792819,11.468312,-2.128222,NaN,NaN,2.396569,-2.069681,-1.775272,0.208388
18658,18658.0,119.0,0.394907,0.390109,0.785016,3.244918,14.149773,10.904855,-1.393431,10.315544,-2.124817,NaN,NaN,2.315878,-1.290539,-3.199835,0.211264
20458,20458.0,131.0,0.419026,0.380550,0.799576,3.139934,14.158045,11.018111,-1.945749,10.596784,-2.119543,NaN,NaN,2.118335,-1.621137,-3.090126,0.210399
17519,17519.0,112.0,0.410204,0.441248,0.851451,1.437104,12.318039,10.880936,-1.190646,10.619199,-2.117326,NaN,NaN,1.524122,-2.086746,-1.155653,0.207567


In [63]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [64]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [65]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [66]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [67]:
df_sliced=w.result
# df_sliced

In [68]:
best_step=[]
df_test=df.copy()
df_test=df[df.epoch>30]
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1b',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.4,sort_col='chi_spec1',head=4,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.9694477302137197, 'chi_spec1': 1.6224463026809102}
[0, 132]
{'chi_1': 0.9694477302137197, 'chi_spec1': 1.6224463026809102}
[0, 132]
{'chi_1': 0.29680608746610204, 'chi_spec1': 0.571430845395223}
[array([18560, 14160,  5510,  5410]), array([ 5010, 16960]), array([17040, 17300, 20480, 19390])]
[ 5010  5410  5510 14160 16960 17040 17300 18560 19390 20480]


In [69]:
# best_step=[19390]
# best_step=np.arange(32800,32900,10).astype(int)

In [70]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 22)
[(32, 5010), (34, 5410), (35, 5510), (90, 14160), (108, 16960), (109, 17040), (110, 17300), (118, 18560), (124, 19390), (131, 20480)]


In [71]:
df_best[col_list]


,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
500,32-5010,0.108659,1.025119,46.236136,0.101669,0.006989,0.000000,0.000738,3.140218,70.703576,32,5010
540,34-5410,0.063591,1.199828,50.357925,0.043892,0.019699,0.000000,0.000290,4.807547,49.322587,34,5410
550,35-5510,0.062324,0.531866,47.704315,0.043004,0.019321,0.000000,0.000281,8.294994,18.627608,35,5510
1415,90-14160,0.056524,0.421598,44.602628,0.047668,0.008852,0.000004,0.000292,5.221950,35.277506,90,14160
1695,108-16960,0.094000,1.017937,47.022980,0.086997,0.007003,0.000000,0.000641,4.244345,41.262724,108,16960
1703,109-17040,0.083675,0.158329,45.830732,0.032479,0.051181,0.000015,0.000229,9.653751,15.594910,109,17040
1729,110-17300,0.203939,0.163228,48.758086,0.025644,0.178284,0.000011,0.000193,10.419166,7.753285,110,17300
1855,118-18560,0.055009,0.200137,47.076156,0.035422,0.019588,0.000000,0.000204,6.580441,8.617940,118,18560
1938,124-19390,0.065645,0.164665,44.574971,0.034436,0.031209,0.000000,0.000238,7.506971,9.700352,124,19390
2047,131-20480,0.263293,0.163609,46.058059,0.033467,0.229818,0.000007,0.000223,8.122044,9.298131,131,20480


### Plots 2: stored data

In [72]:

def f_plot_hist_spec(df,dict_bkg,plot_type,img_size):

    
    assert plot_type in ['hist','spec','grid','spec_relative','hist_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative','hist_relative']:     fig=plt.figure(figsize=(6,6))


    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            
            
        if plot_type=='hist_relative':

            x2=row.hist_bin_centers
            y2=row.hist_val
            yerr2=row.hist_err
            x2=f_invtransform(x2)
            
#             plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['hist_val'],dict_bkg['hist_err']

            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            plt.xlim(0,img_size/2)
            plt.ylim(0,5)
            plt.legend()

        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            print(images.shape)
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.xlim(0,img_size/2)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)
        plt.legend()
        
    if plot_type=='hist_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [73]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type,img_size)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative','hist_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('32-5010', '34-5410', '35-5510', '90-…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [ ]:
print(s_val.shape)


In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210123_050147_3d_l0.5_80k/images/gen_img_epoch-28_step-71620.npy'
a1=np.load(fname)

In [ ]:
s_val.shape,a1.shape

In [ ]:
f_plot_grid(a1[6:14,:,:,0],cols=4,fig_size=(4,2))

In [ ]:
f_plot_grid(s_val[100:108,:,:,0],cols=4,fig_size=(4,2))

In [ ]:
df_best[col_list]

In [ ]:
dims = [2,3,4,7]
np_x = np.random.uniform(size=dims).astype(np.float32)

In [ ]:
90000/(32*32)

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy'
np.load(fname,mmap_mode='r').shape


In [ ]:
! du -hs /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy

